In [1]:
from math import exp

# Лабораторная №2

## Полезные ссылки

- [метод ньютона](http://fourier.eng.hmc.edu/e176/lectures/ch3/node6.html)

## Метод сопряженных направлений

## Метод Ньютона

## Testing

### Test data

In [3]:
f = [
    lambda x, y: 100*(y-x)**2 + (1-x)**2,
    lambda x, y: 100*(y-x**2)**2 + (1-x)**2,
]

f_max = lambda x, y: 2*exp**(-((x-1)/2)**2 - (y-1)**2) + 3*exp**(-((x-2)/3)**2 - (((y-3)/2)**2))

### Graphics

#### Statistics

#### Trajectory